In [ ]:
test_queries = [
    {"query": "What is the population of Paris?", "type": "simple_fact"},
    {"query": "Compare the public transportation systems of Tokyo and New York City.", "type": "complex_reasoning"},
    {"query": "Describe the architectural features of the Sydney Opera House.", "type": "multimodal"},
    {"query": "What are the top attractions in Bangkok?", "type": "out_of_scope"},
    {"query": "Recommend a city for a summer vacation.", "type": "open_ended"},
    {"query": "What's the best time to visit Rome?", "type": "temporal"},
    {"query": "How does the cuisine differ between Barcelona and Dubai?", "type": "comparative"},
    {"query": "What are some budget-friendly activities in London?", "type": "specific_aspect"},
    {"query": "Describe the natural landscape around Machu Picchu.", "type": "descriptive"},
    {"query": "What's the historical significance of the Colosseum in Rome?", "type": "historical"}
]


In [ ]:
from llama_index.core.evaluation import (
    ResponseEvaluator,
    DatasetGenerator,
    QueryResponseDataset
)

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM

# 加载本地模型
embed_model_path = "/home/work/var/data/ssr-share-data/m3e-base/"
llm_model_path = "/home/work/var/data/ssr-share-data/Qwen1.5-7B-Chat/"

# 加载文档
documents = SimpleDirectoryReader('temp_docs').load_data()

# 初始化嵌入模型
embed_model = HuggingFaceEmbedding(model_name=embed_model_path)


# 初始化大语言模型
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == 'assistant':
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<|assistant|>\n"

    return prompt

def completion_to_prompt(completion):
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"


llm = HuggingFaceLLM(
    model_name=llm_model_path,
    tokenizer_name=llm_model_path,
    context_window=3900,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95, "pad_token_id": 151645},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="auto",
)


In [ ]:

import nest_asyncio
nest_asyncio.apply()

# 创建索引
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

# 创建查询引擎
query_engine = index.as_query_engine(llm=llm)

# 创建数据集生成器
data_generator = DatasetGenerator.from_documents(documents, llm=llm)

# 生成评估数据集
eval_dataset = data_generator.generate_questions_from_nodes(num=10)

In [7]:
# 创建评估器
evaluator = ResponseEvaluator(llm=llm)

# 创建查询引擎
query_engine = index.as_query_engine(llm=llm)
print(f"query_engine: {query_engine}")

query_engine: <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x7f5858d4ea10>


In [11]:

# 进行评估
eval_results = []
for query_data in test_queries:
    query = query_data["query"]
    response = query_engine.query(query)
    # print(response)
    # 评估响应质量
    eval_result = evaluator.evaluate_response(
        query=query,
        response=response,
        reference=None,  # 在实际应用中，你可能需要提供参考答案
        context=None
    )

    eval_results.append({
        "query": query,
        "type": query_data["type"],
        "response": str(response),
        "evaluation": eval_result
    })


Sorry, but I can't find information about the population of Paris in the given context. The context discusses historical and modern sites in French Guiana and French Southern Territories, not Paris. If you need population data for Paris, you would need to look up recent statistical sources or a specific fact from a different reference.
I'm sorry, but the given context information does not compare the public transportation systems of Tokyo and New York City. The provided text details historical bridges in Macao and a modern building in Benin. If you want to compare these, you would need separate information about the public transport in each city. 

For a comparison between Tokyo and New York City's public transportation systems, you would typically look at factors such as their train networks (Tokyo's bullet trains vs. New York's subway systems), bus systems, accessibility, and any unique features like monorails or ferries. You would find this type of information in travel guides, tran

In [12]:

# 分析评估结果
def analyze_results(results):
    type_scores = {}
    for result in results:
        query_type = result["type"]
        score = result["evaluation"].score
        if query_type not in type_scores:
            type_scores[query_type] = []
        type_scores[query_type].append(score)
  

    print("Evaluation Results:")
    for query_type, scores in type_scores.items():
        avg_score = sum(scores) / len(scores)
        print(f"{query_type}: Average score = {avg_score:.2f}")
  

    overall_avg = sum([r["evaluation"].score for r in results]) / len(results)
    print(f"Overall Average Score: {overall_avg:.2f}")

analyze_results(eval_results)

Evaluation Results:
simple_fact: Average score = 0.00
complex_reasoning: Average score = 0.00
multimodal: Average score = 0.00
out_of_scope: Average score = 0.00
open_ended: Average score = 0.00
temporal: Average score = 0.00
comparative: Average score = 0.00
specific_aspect: Average score = 0.00
descriptive: Average score = 0.00
historical: Average score = 0.00
Overall Average Score: 0.00


In [13]:
def human_evaluation(eval_results):
    for result in eval_results:
        print(f"\nQuery: {result['query']}")
        print(f"Type: {result['type']}")
        print(f"Response: {result['response']}")
        print(f"Automated Score: {result['evaluation'].score}")
      

        human_score = float(input("Please rate the response (0-1): "))
        feedback = input("Any additional feedback? ")
      

        result["human_score"] = human_score
        result["human_feedback"] = feedback
  

    return eval_results

evaluated_results = human_evaluation(eval_results)



Query: What is the population of Paris?
Type: simple_fact
Response: Sorry, but I can't find information about the population of Paris in the given context. The context discusses historical and modern sites in French Guiana and French Southern Territories, not Paris. If you need population data for Paris, you would need to look up recent statistical sources or a specific fact from a different reference.
Automated Score: 0.0

Query: Compare the public transportation systems of Tokyo and New York City.
Type: complex_reasoning
Response: I'm sorry, but the given context information does not compare the public transportation systems of Tokyo and New York City. The provided text details historical bridges in Macao and a modern building in Benin. If you want to compare these, you would need separate information about the public transport in each city. 

For a comparison between Tokyo and New York City's public transportation systems, you would typically look at factors such as their train net

In [14]:
import time
import psutil

def measure_performance(query_engine, queries, num_runs=3):
    performance_data = []

    for query in queries:
        query_times = []
        memory_usage = []

        for _ in range(num_runs):
            start_time = time.time()
            start_memory = psutil.virtual_memory().used

            _ = query_engine.query(query["query"])

            end_time = time.time()
            end_memory = psutil.virtual_memory().used

            query_time = end_time - start_time
            memory_used = end_memory - start_memory

            query_times.append(query_time)
            memory_usage.append(memory_used)

        avg_time = sum(query_times) / num_runs
        avg_memory = sum(memory_usage) / num_runs

        performance_data.append({
            "query": query["query"],
            "type": query["type"],
            "avg_time": avg_time,
            "avg_memory": avg_memory
        })
  

    return performance_data

performance_results = measure_performance(query_engine, test_queries)

# 分析性能结果
def analyze_performance(results):
    type_performance = {}
    for result in results:
        query_type = result["type"]
        if query_type not in type_performance:
            type_performance[query_type] = {"time": [], "memory": []}
        type_performance[query_type]["time"].append(result["avg_time"])
        type_performance[query_type]["memory"].append(result["avg_memory"])

    print("Performance Analysis:")
    for query_type, data in type_performance.items():
        avg_time = sum(data["time"]) / len(data["time"])
        avg_memory = sum(data["memory"]) / len(data["memory"])
        print(f"{query_type}:")
        print(f"  Average Time: {avg_time:.4f} seconds")
        print(f"  Average Memory Usage: {avg_memory / 1024 / 1024:.2f} MB")

    overall_avg_time = sum([r["avg_time"] for r in results]) / len(results)
    overall_avg_memory = sum([r["avg_memory"] for r in results]) / len(results)
    print(f"Overall Average Time: {overall_avg_time:.4f} seconds")
    print(f"Overall Average Memory Usage: {overall_avg_memory / 1024 / 1024:.2f} MB")

analyze_performance(performance_results)


Performance Analysis:
simple_fact:
  Average Time: 3.3682 seconds
  Average Memory Usage: -20.48 MB
complex_reasoning:
  Average Time: 5.0689 seconds
  Average Memory Usage: 7.36 MB
multimodal:
  Average Time: 4.5192 seconds
  Average Memory Usage: 5.48 MB
out_of_scope:
  Average Time: 2.0805 seconds
  Average Memory Usage: 59.75 MB
open_ended:
  Average Time: 3.7268 seconds
  Average Memory Usage: -16.97 MB
temporal:
  Average Time: 3.2388 seconds
  Average Memory Usage: -48.37 MB
comparative:
  Average Time: 3.0204 seconds
  Average Memory Usage: -778.75 MB
specific_aspect:
  Average Time: 3.4214 seconds
  Average Memory Usage: 833.04 MB
descriptive:
  Average Time: 5.3683 seconds
  Average Memory Usage: 15.83 MB
historical:
  Average Time: 6.7144 seconds
  Average Memory Usage: -54.46 MB
Overall Average Time: 4.0527 seconds
Overall Average Memory Usage: 0.24 MB


In [15]:
def error_analysis(eval_results, performance_results):
    # 合并评估结果和性能结果
    combined_results = []
    for eval_result, perf_result in zip(eval_results, performance_results):
        combined_results.append({**eval_result, **perf_result})

    # 找出得分最低的查询
    lowest_score = min(combined_results, key=lambda x: x["evaluation"].score)
    print(f"Query with lowest score: {lowest_score['query']}")
    print(f"Score: {lowest_score['evaluation'].score}")
    print(f"Response: {lowest_score['response']}")

    # 找出响应时间最长的查询
    slowest_query = max(combined_results, key=lambda x: x["avg_time"])
    print(f"\nSlowest query: {slowest_query['query']}")
    print(f"Average time: {slowest_query['avg_time']:.4f} seconds")

    # 分析不同类型查询的表现
    type_analysis = {}
    for result in combined_results:
        query_type = result["type"]
        if query_type not in type_analysis:
            type_analysis[query_type] = {"scores": [], "times": []}
        type_analysis[query_type]["scores"].append(result["evaluation"].score)
        type_analysis[query_type]["times"].append(result["avg_time"])

    print("\nPerformance by query type:")
    for query_type, data in type_analysis.items():
        avg_score = sum(data["scores"]) / len(data["scores"])
        avg_time = sum(data["times"]) / len(data["times"])
        print(f"{query_type}:")
        print(f"  Average Score: {avg_score:.2f}")
        print(f"  Average Time: {avg_time:.4f} seconds")
  
    # 提出改进建议
    print("\nImprovement suggestions:")
    if lowest_score["evaluation"].score < 0.5:
        print("- Review and enhance the knowledge base for low-scoring queries")
    if slowest_query["avg_time"] > 5:
        print("- Optimize retrieval process for slow queries")
    if any(result["human_score"] - result["evaluation"].score > 0.3 for result in eval_results):
        print("- Refine automated evaluation metrics to better align with human judgment")

error_analysis(evaluated_results, performance_results)


Query with lowest score: What is the population of Paris?
Score: 0.0
Response: Sorry, but I can't find information about the population of Paris in the given context. The context discusses historical and modern sites in French Guiana and French Southern Territories, not Paris. If you need population data for Paris, you would need to look up recent statistical sources or a specific fact from a different reference.

Slowest query: What's the historical significance of the Colosseum in Rome?
Average time: 6.7144 seconds

Performance by query type:
simple_fact:
  Average Score: 0.00
  Average Time: 3.3682 seconds
complex_reasoning:
  Average Score: 0.00
  Average Time: 5.0689 seconds
multimodal:
  Average Score: 0.00
  Average Time: 4.5192 seconds
out_of_scope:
  Average Score: 0.00
  Average Time: 2.0805 seconds
open_ended:
  Average Score: 0.00
  Average Time: 3.7268 seconds
temporal:
  Average Score: 0.00
  Average Time: 3.2388 seconds
comparative:
  Average Score: 0.00
  Average Time: 

In [16]:
import logging
from datetime import datetime

logging.basicConfig(filename='rag_system.log', level=logging.INFO)

def log_query(query, response, performance):
    logging.info(f"""
    Timestamp: {datetime.now()}
    Query: {query}
    Response: {response}
    Response Time: {performance['avg_time']:.4f} seconds
    Memory Usage: {performance['avg_memory'] / 1024 / 1024:.2f} MB
    """)

# 在查询处理函数中使用
def process_query(query):
    start_time = time.time()
    start_memory = psutil.virtual_memory().used
  
    response = query_engine.query(query)
  
    end_time = time.time()
    end_memory = psutil.virtual_memory().used
  
    performance = {
        "avg_time": end_time - start_time,
        "avg_memory": end_memory - start_memory
    }
  
    log_query(query, str(response), performance)
  
    return response

# 示例使用
test_query = "What are the must-visit attractions in Paris?"
result = process_query(test_query)
print(result)


Batches: 100%|██████████| 1/1 [00:00<00:00, 126.34it/s]


I'm sorry, but the given context does not mention any attractions in Paris. The first reference is about French Guiana's historical site in Clarkport, while the second discusses modern architecture in Melissabury, which is part of French Southern Territories. For must-visit attractions in Paris, you would need to refer to information specific to that city.
